In [64]:
#Update your token
STUDENT_TOKEN = 'EUGENIOMARCHIORI'

Server communication game and game_loop.

In [65]:
## ignore this code, just used for submission
import requests
import pprint
import json
import random
import time
from copy import copy, deepcopy

class Game:
  def __init__(self, state, status, player):
    self.state = state
    self.status = status
    self.player = player

  def is_waiting(self):
    return self.status == 'waiting'

  def is_end(self):
    return self.status == 'complete'

  def get_board(self):
    print(self.state)
    return json.loads(self.state)

  def actions(self):
    return []

  def print_game(self):
    print(self.state)

def new_game(game_type, multi_player = False):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/new-game?TOKEN=%s&game-type=%s&multi-player=%s' % (STUDENT_TOKEN, game_type, 'True' if multi_player else 'False'))
    if r.status_code == 200:
      return r.json()['game-id']
    print(r.content)

def join_game(game_type, game_id):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/join-game?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return r.json()['player']
    print(r.content)

def game_state(game_type, game_id, GameClass):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/game-state?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return GameClass(r.json()['state'], r.json()['status'], r.json()['player'])
    print(r.content)

def update_game(game_type, game_id, player, move):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/update-game?TOKEN=%s&game-type=%s&game-id=%s&player=%s&move=%s' % (STUDENT_TOKEN, game_type, game_id, player, move))
    if r.status_code == 200:
      return r.content
    print(r.content)

def game_loop(solver, GameClass, game_type, multi_player = False, id = None):
  while id == None:
    print('\033[92mCreating new game...\033[0m')
    id = new_game(game_type, multi_player)

  print('\033[92mJoining game with id: %s\033[0m' % id)
  player = join_game(game_type, id)

  print('\033[92mPlaying as %s\033[0m' % player)

  game = game_state(game_type, id, GameClass)
  print('\033[91mWaiting for the other player to join...\033[0m')
  while game.is_waiting():
    time.sleep(10)
    game = game_state(game_type, id, GameClass)

  while True:
    game = game_state(game_type, id, GameClass)
    game.print_game()
    if game.is_end():
      if game.player == '-':
        print('\033[94mdraw\033[0m')
      else:
        print('\033[92mYou won\033[0m' if game.player == player else '\033[91mYou lost\033[0m')
      return
    if game.player == player:
      print('Making next move...')
      move = solver(game)
      update_result = update_game(game_type, id, player, json.dumps(move))
      print(update_result)
    else:
      time.sleep(2)

Random solver

In [66]:
def random_solver(game):
  if game.state['phase'] == "place":
    if game.player == 'X':
      positions = [(y, x) for x in range(2, 6) for y in range(2)]
    else:
      positions = [(7 - y, x) for x in range(2, 6) for y in range(2)]
    pieces = ['F', 'G', 'M', 'S', 'S', 's', 'B', 'f']
    random.shuffle(pieces)
    final_pos = []
    for pos in positions:
      final_pos.append((pos[0], pos[1], pieces.pop()))
    print(final_pos)
    return final_pos
  else:
    return random.choice(game.actions())

Game specific code

In [67]:
from functools import reduce
from copy import copy, deepcopy
import json


class Stratego(Game):
  def __init__(self, state, status, player):
    Game.__init__(self, state, status, player)

  def actions(self):
    return self.state['possible_actions']

  def card_str(self, card):
    return '+'.join(map(str, card))

  def print_game(self):
    print('Board: ' + "\n".join("".join(row) for row in self.state['board']))
    if self.state['past_actions']:
      print('Last action: ' + str(self.state['past_actions'][-1]))
      print('Last two actions: ' + " ".join(str(action) for action in self.state["past_actions"][-2:]))

  def other_player(self):
    if self.player == 'X': return 'O'
    if self.player == 'O': return 'X'

In [68]:
game_loop(random_solver, Stratego, 'stratego', multi_player=False, id=None)

Creating new game...
Joining game with id: 14440
Playing as O
Waiting for the other player to join...
Board: __XXXX__
__XXXX__
________
__L__L__
__L__L__
________
________
________
Making next move...
[(7, 2, 'S'), (6, 2, 'B'), (7, 3, 'f'), (6, 3, 'G'), (7, 4, 'S'), (6, 4, 's'), (7, 5, 'F'), (6, 5, 'M')]
b'Valid move'
Board: __XXXX__
_X_XXX__
________
__L__L__
__L__L__
________
__BGsM__
__SfSF__
Last action: ['X', [1, 2], [1, 1], 'move']
Last two actions: ['X', [1, 2], [1, 1], 'move']
Making next move...
b'Valid move'
Board: __XXXX__
_X__XX__
________
__LX_L__
__L__L__
________
__BGsM__
__SfS_F_
Last action: ['X', [1, 3], [3, 3], 'move']
Last two actions: ['O', [7, 5], [7, 6], 'move'] ['X', [1, 3], [3, 3], 'move']
Making next move...
b'Valid move'
Board: __XXXX__
_X__XX__
________
__L_XL__
__L__L__
________
__BGs___
__SfSMF_
Last action: ['X', [3, 3], [3, 4], 'move']
Last two actions: ['O', [6, 5], [7, 5], 'move'] ['X', [3, 3], [3, 4], 'move']
Making next move...
b'Valid move'
Board: _